In [1]:
#===Imports====#
import pandas as pd
import sqlalchemy
import numpy as np
import sqlite3
#=============#

In [2]:
conn = sqlite3.connect("D:\\ALAM\\CN-Intermodal verified.db")
#engine = sqlalchemy.create_engine("sqlite:///CN-Intermodal verified.db")
Carriers = pd.read_sql("select * from carriers;",conn)
trains = pd.read_sql('select * from trains;',conn)
cars = pd.read_sql('select * from cars;',conn)
containers = pd.read_sql('select * from containers;',conn)
platforms = pd.read_sql('select * from platforms;',conn)
Carriers = Carriers.dropna().drop('id',axis=1)
Carriers_modified = pd.merge(Carriers,cars[['id','loco_flag','truck_center_length','num_platforms','outside_length']],left_on='car_id',right_on='id',how='left').drop('id',axis=1)
Carriers_modified = pd.merge(Carriers_modified,containers[['id','length']],left_on='container_id',right_on='id',how='left').drop('id',axis=1)
Carriers_modified = pd.merge(Carriers_modified,platforms[['id','platform_interior_length']],left_on='platform_id',right_on='id',how='left').drop('id',axis=1)
Carriers_modified = Carriers_modified[~((Carriers_modified.slot!='TOP') & (Carriers_modified.length==53))]

Carriers_modified = Carriers_modified[['train_id', 'train_num','car_id', 'platform_id','container_id', 'slot','seq_num','loco_flag','length','truck_center_length','platform_interior_length','num_platforms','outside_length']]
Carriers_modified = Carriers_modified.reset_index()
df = Carriers_modified.copy(deep=True)

# Filter by TrainId

In [3]:
df_new = df[df['train_id']==3]
df_new1 = df_new.groupby('platform_id',as_index=False).agg({'container_id':'nunique'})
df_new1.columns=['platform_id','nuniquecontainers']
df_new2 = df_new.groupby('platform_id',as_index=False).agg({'container_id':'size'})
df_new2.columns=['platform_id','sizecontainers']
df_new3 = df_new.groupby('platform_id',as_index=False).agg({'slot':lambda X : True if 'TOP' in set(X) else False})
df_new3.columns = ['platform_id','TOP_FLAG']
df_new = df_new1.merge(df_new2,on='platform_id',how='left')
df_new = df_new.merge(df_new3,on='platform_id',how='left')

In [4]:
df_new

,platform_id,nuniquecontainers,sizecontainers,TOP_FLAG
0,86.0,2,3,True
1,87.0,2,3,True
2,88.0,2,3,True
3,89.0,2,3,True
4,90.0,2,3,True
...,...,...,...,...
162,286.0,1,2,False
163,289.0,1,2,False
164,290.0,1,2,False
165,292.0,1,2,False


In [5]:
#===========Encodings=================#
''''''
def encode_row(X):
    '''Method to Encode Row'''
    encoding = None 
    if (X['nuniquecontainers']==3 and X['sizecontainers']==3):
        encoding = 1 # All 3 Unique Containers
    elif (X['nuniquecontainers']==2 and X['sizecontainers']==3 and X['TOP_FLAG']):
        encoding = 2 # Left, Right 40 Top Filled
    elif (X['nuniquecontainers']==2 and X['sizecontainers']==2 and not(X['TOP_FLAG'])):
        encoding = 3 # Left, Right 20-20 Top Empty
    elif (X['nuniquecontainers']==1 and X['sizecontainers']==2):
        encoding = 4 # Left,Right 40 Top empty
    elif (X['nuniquecontainers']==1 and X['sizecontainers']==1):
        encoding = 5 #Either Left or right
    else:
        encoding = 0
    return encoding
#======================================#

df_new['Encodings'] = df_new.apply(encode_row,axis=1)

    
        

In [6]:
df_new

,platform_id,nuniquecontainers,sizecontainers,TOP_FLAG,Encodings
0,86.0,2,3,True,2
1,87.0,2,3,True,2
2,88.0,2,3,True,2
3,89.0,2,3,True,2
4,90.0,2,3,True,2
...,...,...,...,...,...
162,286.0,1,2,False,4
163,289.0,1,2,False,4
164,290.0,1,2,False,4
165,292.0,1,2,False,4


In [7]:
df_new['platform_id'].nunique()

167

In [8]:
df_new1=Carriers_modified['platform_id'].nunique()

In [9]:
df_new1

40416

In [10]:
df[df.train_id==3]['container_id'].nunique()

295

In [11]:
from collections import defaultdict


def genagg(df,train_id):
    df = df.copy(deep=True)
    df = df[df['train_id']==train_id]
    platforms,containers = list(df.platform_id.unique()),list(df.container_id.unique())
    df_subset_length = df[['container_id','length']]
    df_subset_length = df_subset_length.drop_duplicates('container_id')
    container_len_dict = dict(zip(df_subset_length.container_id,df_subset_length.length))
    platform_fill_status = {i:{'left':None,'right':None,'top':None} for i in platforms}
    platform_container_mapping = defaultdict(list)
    added_list = []
    for k,v in container_len_dict.items():
        if not(k in added_list):
            while True:
                plat = np.random.choice(platforms,1)[0]
                if v==40:
                    if ((platform_fill_status[plat]['left']==None) and (platform_fill_status[plat]['right']==None)):
                        added_list.append(k)
                        platform_container_mapping[plat].append(k)
                        platform_fill_status[plat]['left'] = 'filled'
                        platform_fill_status[plat]['right'] = 'filled'
                        break
                    elif ((platform_fill_status[plat]['left']!=None) and (platform_fill_status[plat]['right']!=None) and (platform_fill_status[plat]['top']==None)):
                        added_list.append(k)
                        platform_container_mapping[plat].append(k)
                        platform_fill_status[plat]['left'] = 'filled'
                        platform_fill_status[plat]['right'] = 'filled'
                        platform_fill_status[plat]['top'] = 'filled'
                        break
                    else:
                        pass
                
                elif v<40:
                    if ((platform_fill_status[plat]['left']!=None) and (platform_fill_status[plat]['right']==None)):
                        added_list.append(k)
                        platform_container_mapping[plat].append(k)
                        platform_fill_status[plat]['right'] = 'filled'
                        break
                    elif ((platform_fill_status[plat]['left']==None) and (platform_fill_status[plat]['right']!=None)):
                        added_list.append(k)
                        platform_container_mapping[plat].append(k)
                        platform_fill_status[plat]['left'] = 'filled'
                        break
                    elif ((platform_fill_status[plat]['left']==None) and (platform_fill_status[plat]['right']==None)):
                        added_list.append(k)
                        platform_container_mapping[plat].append(k)
                        platform_fill_status[plat]['left'] = 'filled'
                        break
                    else:
                        pass
                
                else:
                    if ((platform_fill_status[plat]['left']!=None) and (platform_fill_status[plat]['right']!=None)):
                        added_list.append(k)
                        platform_container_mapping[plat].append(k)
                        platform_fill_status[plat]['top'] = 'filled'
                        break
                    else:
                        pass
                        
    return platform_fill_status,platform_container_mapping,added_list
    
    

In [38]:
#pfstatus,pcmap,addlist = genagg(df,3)

def generate_chromosomes(tid,n_chromosomes=10):
    '''Method to Generate Chromosomes'''
    Chromosome_list = []
    for _ in range(n_chromosomes):
        pfstatus,pcmap,addlist = genagg(df,tid)
        chromosome_1 = []
        for k,v in pcmap.items():
            for _v in v:
                chromosome_1.append([k,_v])

        chromosome_1 = pd.DataFrame(chromosome_1,columns=['platform_id','container_id'])
        d1 = chromosome_1.groupby('platform_id',as_index=False).agg({'container_id':'nunique'})
        d2 = chromosome_1.groupby('platform_id',as_index=False).agg({'container_id':'size'})
        d1.columns,d2.columns = ['platform_id','nuniquecontainers'],['platform_id','sizecontainers']
        d = d1.merge(d2,how='left',on='platform_id')
        d['TOP_FLAG'] = d['platform_id'].apply(lambda X : True if pfstatus[X]['top']=='filled' else False)
        d['Encodings'] = d.apply(encode_row,axis=1)
        Chromosome_list.append(d)
    
    return Chromosome_list
        

        
        

        

In [39]:
chromosome_li =generate_chromosomes(3,25)

In [40]:
print(chromosome_li)

[     platform_id  nuniquecontainers  sizecontainers  TOP_FLAG  Encodings
0           86.0                  2               2      True          0
1           87.0                  2               2      True          0
2           90.0                  2               2      True          0
3           91.0                  2               2      True          0
4           92.0                  2               2      True          0
..           ...                ...             ...       ...        ...
155        286.0                  2               2      True          0
156        289.0                  3               3      True          1
157        290.0                  1               1     False          5
158        292.0                  3               3      True          1
159        293.0                  2               2      True          0

[160 rows x 5 columns],      platform_id  nuniquecontainers  sizecontainers  TOP_FLAG  Encodings
0           86.0         

In [64]:
def cross_over(father, mother):
    '''Method to implement Ordered Cross over '''
    i = np.random.choice(range(len(father)), 2)
    i = sorted(i)
    child_1, child_2 = ["_" for k in father], ["_" for k in mother]
    child_2[i[0]:(i[1] + 1)], child_1[i[0]:(i[1] + 1)] = father[i[0]:(i[1] + 1)], mother[i[0]:(i[1] + 1)]
    for idx, k in enumerate(child_2):
        if k == "_":
            for j in mother:
                if not (j in child_2):
                    child_2[idx] = j
                    break

    for idx, k in enumerate(child_1):
        if k == "_":
            for j in father:
                if not (j in child_1):
                    child_1[idx] = j
                    break
    child1_m,child2_m = [np.random.randint(0,6,1)[0] if k=='_' else k for k in child_1],[np.random.randint(0,6,1)[0] if k=='_' else k for k in child_2]
    return child1_m,child2_m

In [65]:
#============CrossOvers=========================#
def get_crossed(chromosomes,nchildren=100):
    new_generation = []
    for _ in range(int(nchildren/2)):
        d1,d2 = np.random.randint(0,len(chromosomes),2)
        d1,d2 = chromosomes[d1],chromosomes[d2]
        c1,c2 = cross_over(d1.Encodings,d2.Encodings)
        d1,d2 = d1.drop('Encodings',axis=1),d2.drop('Encodings',axis=1)
        d1['Encodings'],d2['Encodings'] = pd.Series(c1),pd.Series(c2)
        new_generation.extend([d1,d2])
    return new_generation

next_generation = get_crossed(chromosome_li)

In [66]:
next_generation

[     platform_id  nuniquecontainers  sizecontainers  TOP_FLAG  Encodings
 0           86.0                  2               2      True          3
 1           88.0                  2               2      True          5
 2           89.0                  2               2      True          1
 3           90.0                  1               1     False          2
 4           91.0                  1               1     False          0
 ..           ...                ...             ...       ...        ...
 157        286.0                  2               2      True          2
 158        289.0                  2               2      True          2
 159        290.0                  2               2      True          2
 160        292.0                  4               4      True          4
 161        293.0                  2               2      True          0
 
 [162 rows x 5 columns],
      platform_id  nuniquecontainers  sizecontainers  TOP_FLAG  Encodings
 0         